In [1]:
import pandas as pd
import numpy as np
import os
import trimesh
from meshparty import trimesh_io, trimesh_vtk, skeleton, utils
import vtk
import random

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# Load astrocyte cellids into a list

astrocytes = pd.read_csv('data/astrocytes.csv', index_col=[0])
astro_list = astrocytes.cell_segid.to_list()
len(astro_list)

44

In [4]:
astrocytes.head()

,centroid_x,centroid_y,centroid_z,cell_segid,cell_type,cell_subtype
0,88240,60656,256,648518346349517321,glia,astrocyte
1,72368,56912,590,648518346341392909,glia,astrocyte
4,72704,54688,506,648518346349536799,glia,astrocyte
5,74576,75520,1163,648518346349525537,glia,astrocyte
6,79328,67360,445,648518346349525544,glia,astrocyte


In [5]:
# create vasculature list

celltypes = pd.read_csv("data/220309_cell_type_classification.csv", index_col=[0])
vasc = celltypes[celltypes.cell_type == 'vasculature']
vasc_list = vasc.cell_segid.to_list()
len(vasc_list)

72

In [6]:
# this csv file is 268 MB and too large to place on Github
# in order to run this notebook, you must first download the file and place on your local machine in the /data folder
# the csv file is available here: https://zenodo.org/record/5579388/files/211019_mitochondria_info.csv

mito = pd.read_csv('data/211019_mitochondria_info.csv')
mito.head()

,mito_id,mito_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,bbox_beg_x_vx,bbox_beg_y_vx,bbox_beg_z_vx,bbox_end_x_vx,bbox_end_y_vx,bbox_end_z_vx,cellid,ctr_pos_x_nm,ctr_pos_y_nm,ctr_pos_z_nm
0,3384540,5916,103764,47040,103,103734,47016,102,103798,47066,107,648518346348124201,371475.12,168403.20,4120
1,2526419,1075376,87582,60964,1435,87204,59752,1215,87992,62504,1609,648518346346303282,313543.56,218251.12,57400
2,1379480,483500,65740,73550,392,65556,73322,292,65976,73880,509,648518346341355778,235349.20,263309.00,15680
3,3380073,23140,103750,46904,176,103704,46862,173,103812,46946,181,648518346348124771,371425.00,167916.32,7040
4,1783610,11996,75124,43042,2100,75088,43008,2097,75164,43086,2103,648518346342925287,268943.92,154090.36,84000


In [7]:
mito_list = mito[mito['cellid'].isin(astro_list)].mito_id.values.tolist()
len(mito_list)

41843

In [8]:
mito_list_dictionary_by_astro = {
    f'astro_{uid}': mito[mito.cellid == uid].mito_id.tolist()
    for uid in astro_list
}

# print(mito_list_dictionary_by_astro)

In [9]:
type(mito_list_dictionary_by_astro)

dict

In [10]:
mesh_dir = 'data/neuron_meshes_v185/dec/' # or change to your desired folder
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mm = trimesh_io.MeshMeta(cv_path=seg_source,
                         disk_cache_path=mesh_dir, 
                         cache_size=20)

# setup the mesh meta to handle downloads and caching
mito_mesh_dir = 'data/meshes/dec/'
mito_source = "precomputed://https://td.princeton.edu/sseung-archive/pinky100-mito/seg_191220"
mito_mm = trimesh_io.MeshMeta(cv_path=mito_source,
                         disk_cache_path=mito_mesh_dir)

In [11]:
# generate trimesh objects of vascular and astro cells and place in a dictionary

def update_mesh_dictionary(cell_list, mesh_dictionary, prefix, mesh_dir):
    for cellid in cell_list:
        mesh_file = os.path.join(mesh_dir, str(cellid) + '.ply')
        mesh_dictionary[prefix + str(cellid)] = trimesh.load_mesh(mesh_file)

# Initialize the mesh dictionary
mesh_dictionary = {}

# Update the dictionary with astro_list
update_mesh_dictionary(astro_list, mesh_dictionary, 'astro_', mesh_dir)

# Update the dictionary with vasc_list
update_mesh_dictionary(vasc_list, mesh_dictionary, 'vasc_', mesh_dir)


In [12]:
len(mesh_dictionary)

61

In [13]:
# mito_list[:10]

In [14]:
# Create the nested dictionary of decimated mito meshes

trimesh_dictionary_by_astro = {}

for astro_key, mito_list in mito_list_dictionary_by_astro.items():
    # Initialize the second-level dictionary
    nested_dict = {}
    
    for mitoid in mito_list:
        # Construct the file path for the ply file
        ply_file = os.path.join(mito_mesh_dir, str(mitoid) + '.ply')
        
        # Debug: Print the file path to check correctness
        # print(f'Processing file: {ply_file}')
        
        # Check if the file exists and is accessible
        if not os.path.isfile(ply_file):
            print(f'Error: File does not exist - {ply_file}')
            continue
        
        try:
            # Load the ply file as a trimesh object
            mito_mesh = trimesh.load_mesh(ply_file)
            # Define the key for the second-level dictionary
            nested_key = f'trimesh_mito_{mitoid}'
            # Store the trimesh object in the nested dictionary
            nested_dict[nested_key] = mito_mesh
        except Exception as e:
            print(f'Error processing file {ply_file}: {e}')
    
    # Store the nested dictionary in the first-level dictionary
    trimesh_dictionary_by_astro[astro_key] = nested_dict

# Now, trimesh_dictionary_by_astro contains the organized trimesh objects


In [15]:
len(trimesh_dictionary_by_astro)

44

In [16]:
# Display the first 10 items in the nested dictionary

item_count = 0
for astro_key, nested_dict in trimesh_dictionary_by_astro.items():
    print(f'{astro_key}:')
    for nested_key, mito_mesh in nested_dict.items():
        print(f'  {nested_key}: {mito_mesh}')
        item_count += 1
        if item_count >= 10:
            break
    if item_count >= 10:
        break


astro_648518346349517321:
  trimesh_mito_2652934: <trimesh.Trimesh(vertices.shape=(3, 3), faces.shape=(1, 3))>
  trimesh_mito_2767244: <trimesh.Trimesh(vertices.shape=(44, 3), faces.shape=(84, 3))>
  trimesh_mito_2645886: <trimesh.Trimesh(vertices.shape=(55, 3), faces.shape=(105, 3))>
  trimesh_mito_2631346: <trimesh.Trimesh(vertices.shape=(224, 3), faces.shape=(451, 3))>
  trimesh_mito_2748362: <trimesh.Trimesh(vertices.shape=(47, 3), faces.shape=(90, 3))>
  trimesh_mito_2741993: <trimesh.Trimesh(vertices.shape=(167, 3), faces.shape=(323, 3))>
  trimesh_mito_2647004: <trimesh.Trimesh(vertices.shape=(45, 3), faces.shape=(86, 3))>
  trimesh_mito_2761178: <trimesh.Trimesh(vertices.shape=(128, 3), faces.shape=(253, 3))>
  trimesh_mito_2532323: <trimesh.Trimesh(vertices.shape=(41, 3), faces.shape=(78, 3))>
  trimesh_mito_2847413: <trimesh.Trimesh(vertices.shape=(58, 3), faces.shape=(112, 3))>


In [17]:
# Render all astros with the same settings and pull the viewpoint centroid from an index in astro_list (default index [0])

# Define colors and opacities
astro_color = [0.5, 0.5, 0.5]  # Neutral gray
astro_opac = 0

vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.1

mito_opac = 0.25

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
for cellid in astro_list:
    cell_key = 'astro_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=astro_opac, 
            color=astro_color
        )

for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )

locals().update(mesh_actor)

# Creating a camera object and defining the view
if 'astro_' + str(astro_list[0]) in mesh_dictionary:
    camera = trimesh_vtk.oriented_camera(mesh_dictionary['astro_' + str(astro_list[0])].centroid, backoff=400)
else:
    print(f"First cell key astro_{astro_list[0]} not found in mesh dictionary.")

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000012D67477450) at 0x0000012DAD1C9CA8>

In [18]:
# Render all astros with the same settings and calculate a global mean of the viewpoint centroid

# Define colors and opacities
astro_color = [0.5, 0.5, 0.5]  # Neutral gray
astro_opac = 0.005

vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.1

mito_opac = 0.25

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

for cellid in astro_list:
    cell_key = 'astro_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=astro_opac, 
            color=astro_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000012D6747B9C0) at 0x0000012D5C8A3EE8>

In [19]:
# Render vasculature and all astros, using random astro colors and a range of astro opacities

# Define colors and opacities for vasc_list
vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.1

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

for cellid in astro_list:
    cell_key = 'astro_' + str(cellid)
    if cell_key in mesh_dictionary:
        # Randomize astro_color and astro_opac for each mesh
        astro_color = np.random.random(size=3)  # Random color
        astro_opac = np.random.uniform(0.01, 0.075)  # Random opacity between 0.005 and 0.05
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=astro_opac, 
            color=astro_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000012D674767B0) at 0x0000012D5C8A3C48>

In [20]:
# Render one astro and all the vasculature 

# Define colors and opacities for vasc_list
vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.1

# Define the specific astro cell ID and its properties
specific_astro_id = 648518346349527319
astro_color = np.random.random(size=3)  # Random color
astro_opac = 0.25

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

# Add the specific astro mesh
astro_key = 'astro_' + str(specific_astro_id)
if astro_key in mesh_dictionary:
    mesh_actor[astro_key] = trimesh_vtk.mesh_actor(
        mesh_dictionary[astro_key], 
        opacity=astro_opac, 
        color=astro_color
    )
    # Use the centroid of this specific astro mesh for the camera
    overall_centroid = mesh_dictionary[astro_key].centroid

# Add all the vasc meshes
for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=150)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000012D6747B370) at 0x0000012DAD1F1EE8>

In [21]:
# Render the vasc and one astro by querying the astro list

# Define colors and opacities for vasc_list
vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.1

# User sets the index value
indx = 0  # Modify this index value as needed

# Get the specific astro ID from the astro_list
specific_astro_id = astro_list[indx]

# Define the specific astro cell properties
astro_color = np.random.random(size=3)  # Random color
astro_opac = 0.25

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

# Add the specific astro mesh
astro_key = 'astro_' + str(specific_astro_id)
if astro_key in mesh_dictionary:
    mesh_actor[astro_key] = trimesh_vtk.mesh_actor(
        mesh_dictionary[astro_key], 
        opacity=astro_opac, 
        color=astro_color
    )
    # Use the centroid of this specific astro mesh for the camera
    overall_centroid = mesh_dictionary[astro_key].centroid

# Add all the vasc meshes
for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

locals().update(mesh_actor)

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=150)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000012D674780F0) at 0x0000012DAD1F19A8>

# render decimated mitochondria meshes

In [22]:
# Render vasc with all astrocyte mitochondria
# astro cell bodies commented out
# even so, vtk view is heavily bogged down 

# Define colors and opacities
astro_color = [0.5, 0.5, 0.5]  # Neutral gray
astro_opac = 0

vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.1

mito_opac = 0.25

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

# uncomment to render astro cell bodies
# for cellid in astro_list:
#     cell_key = 'astro_' + str(cellid)
#     if cell_key in mesh_dictionary:
#         mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
#             mesh_dictionary[cell_key], 
#             opacity=astro_opac, 
#             color=astro_color
#         )
#         centroids.append(mesh_dictionary[cell_key].centroid)

for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

# Creating a new dictionary to hold the mito mesh actors
mito_mesh_actor = {}

# Randomly assign colors to each group of astro meshes and add them to the actors
for astro_key, nested_dict in trimesh_dictionary_by_astro.items():
    # Generate a random color for this group of astro meshes
    astro_random_color = [random.random(), random.random(), random.random()]
    
    for mitoid, mito_mesh in nested_dict.items():
        # Define the key for the mesh actor
        actor_key = f'{astro_key}_trimesh_mito_{mitoid}'
        
        # Check if the loaded object is a Scene
        if isinstance(mito_mesh, trimesh.Scene):
            # Extract geometries from the Scene
            for name, geom in mito_mesh.geometry.items():
                mito_mesh = geom
                # Create the mesh actor with the random color and opacity
                mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                    mito_mesh, 
                    opacity=mito_opac, 
                    color=astro_random_color
                )
                centroids.append(mito_mesh.centroid)
        else:
            # Create the mesh actor with the random color and opacity
            mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                mito_mesh, 
                opacity=mito_opac, 
                color=astro_random_color
            )
            centroids.append(mito_mesh.centroid)

# Merge the mito mesh actors into the main mesh_actor dictionary
mesh_actor.update(mito_mesh_actor)

locals().update(mesh_actor)

# Calculate the overall centroid
if centroids:
    overall_centroid = np.mean(centroids, axis=0)
else:
    overall_centroid = np.array([0, 0, 0])  # Fallback if no centroids are found

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=400)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000013030A1B140) at 0x0000012DAD1F16A8>

In [23]:
# save image to disk

import datetime

save_dir = 'vtk_images/'
base_name = 'all_astro_mito_with_vasc'
extension = 'png'

date_time_stamp = datetime.datetime.now().strftime("%Y_%m_%d_%H%M_%S")
filename = f"{save_dir}{base_name}_{date_time_stamp}.{extension}"

# uncomment to save

# camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=300) # 400 default
# trimesh_vtk.render_actors(mesh_actor.values(),
#                           filename=filename,
#                           do_save=True,
#                           scale=6,
#                           camera=camera
#                          )

In [24]:
# render vasc and one queried astro with all its mitochondria

# Define colors and opacities for vasc_list
vasc_color = [0.5, 0.0, 0.0]  # Dark red
vasc_opac = 0.05

# Define the mito opacity
mito_opac = 0.75

# User sets the index value
indx = 6  # Modify this index value as needed

# Get the specific astro ID from the astro_list
specific_astro_id = astro_list[indx]
print(f"Index: {indx}, Specific Astro ID: {specific_astro_id}")  # Debug statement

# Define the specific astro cell properties
astro_color = np.random.random(size=3)  # Random color
astro_opac = 0.05

# Creating mesh actors with specified colors and opacities without modifying the mesh_dictionary
mesh_actor = {}
centroids = []

# Add the specific astro mesh
astro_key = 'astro_' + str(specific_astro_id)
if astro_key in mesh_dictionary:
    mesh_actor[astro_key] = trimesh_vtk.mesh_actor(
        mesh_dictionary[astro_key], 
        opacity=astro_opac, 
        color=astro_color
    )
    # Use the centroid of this specific astro mesh for the camera
    overall_centroid = mesh_dictionary[astro_key].centroid

# Add all the vasc meshes
for cellid in vasc_list:
    cell_key = 'vasc_' + str(cellid)
    if cell_key in mesh_dictionary:
        mesh_actor[cell_key] = trimesh_vtk.mesh_actor(
            mesh_dictionary[cell_key], 
            opacity=vasc_opac, 
            color=vasc_color
        )
        centroids.append(mesh_dictionary[cell_key].centroid)

# Creating a new dictionary to hold the mito mesh actors
mito_mesh_actor = {}

# Randomly assign colors to each group of astro meshes and add them to the actors
astro_key = f'astro_{specific_astro_id}'
if astro_key in trimesh_dictionary_by_astro:
    for mitoid, mito_mesh in trimesh_dictionary_by_astro[astro_key].items():
        # Generate a random color for this group of astro meshes
        astro_random_color = [random.random(), random.random(), random.random()]
        
        # Define the key for the mesh actor
        actor_key = f'{astro_key}_trimesh_mito_{mitoid}'
        
        # Check if the loaded object is a Scene
        if isinstance(mito_mesh, trimesh.Scene):
            # Extract geometries from the Scene
            for name, geom in mito_mesh.geometry.items():
                mito_mesh = geom
                # Create the mesh actor with the random color and opacity
                mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                    mito_mesh, 
                    opacity=mito_opac, 
                    color=astro_random_color
                )
                centroids.append(mito_mesh.centroid)
        else:
            # Create the mesh actor with the random color and opacity
            mito_mesh_actor[actor_key] = trimesh_vtk.mesh_actor(
                mito_mesh, 
                opacity=mito_opac, 
                color=astro_random_color
            )
            centroids.append(mito_mesh.centroid)

# Merge the mito mesh actors into the main mesh_actor dictionary
mesh_actor.update(mito_mesh_actor)

locals().update(mesh_actor)

# Creating a camera object and defining the view
camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=150)

# Render the actors, will open a pop-up Python window
trimesh_vtk.render_actors(mesh_actor.values(), camera=camera)


Index: 6, Specific Astro ID: 648518346349490239
setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x0000013050E22CE0) at 0x0000012DAD1DD7C8>

In [25]:
# save image to disk

import datetime

save_dir = 'vtk_images/'
base_name = f"astro_{specific_astro_id}_mito_with_vasc"
extension = 'png'

date_time_stamp = datetime.datetime.now().strftime("%Y_%m_%d_%H%M_%S")
filename = f"{save_dir}{base_name}_{date_time_stamp}.{extension}"

# uncomment to save

# camera = trimesh_vtk.oriented_camera(overall_centroid, backoff=150) # 150 default
# trimesh_vtk.render_actors(mesh_actor.values(),
#                           filename=filename,
#                           do_save=True,
#                           scale=6,
#                           camera=camera
#                          )